### aim: 

### date: 

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [3]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE')
#list of models
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS')
import lom
import utils as ut

%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload
import matplotlib.path as mpath
import glob
import pickle
import pandas as pd
import seawater
import time
plt.rcParams.update({'font.size': 12})
font = {'family' : 'normal',
'weight' : 'normal',
'size'   : 12}

plt.rc('font', **font)

In [4]:
import pymannkendall as mk
from scipy import stats

def give_trends(ts_x,ts_y,ystart,yend):

    trend, h, mk_p, z, Tau, s, var_s, mk_slope, intercept = mk.original_test(ts_y)
    lin_slope, intercept, r_value, lin_p, std_err = stats.linregress(ts_x,ts_y)
    
    return mk_slope, mk_p, lin_slope, lin_p

    
def give_summary_1950_2100(wspd, verbose = False):
    
    data = np.zeros([5,8])
    s_y = [1950,1950,2000,2050]
    e_y = [2099,1999,2049,2099]
    tseas = ['FY','DJF','MAM','JJA','SON']

    for y in range(0,4):
        for s in range(0,5):
            t1 = s_y[y]
            t2 = e_y[y]
            seas = tseas[s]

            if seas == 'FY':
                w = wspd.wspd10m.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-12-31'))\
                .groupby('time_counter.year').mean()
            else:
                w = wspd.wspd10m.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-12-31'))
                w = w.sel(time_counter=(w['time_counter.season'] == seas)).groupby('time_counter.year').mean()

            mk_slope, mk_p, lin_slope, lin_p = give_trends(w.year,w,t1,t2)
            mk_slope = mk_slope*10 #decadal
            lin_slope = lin_slope*10 #decadal
            if mk_p > 0.05:
                mk_slope = -999
            if lin_p > 0.05:
                lin_slope = -999  
            if verbose:
                print(f'winds, {seas}, {t1}-{t2}, MK: {mk_slope}, lin: {lin_slope}')
            
            data[s,y*2] = mk_slope
            data[s,y*2+1] = lin_slope
        if verbose:
            print()
        
    df = pd.DataFrame(data, columns=['MK-1950-2100','lin-1950-2100',\
                                   'MK-1950-2000','lin-1950-2000',\
                                   'MK-2000-2050','lin-2000-2050',\
                                   'MK-2050-2100','lin-2050-2100',\
                                   ],\
              index  = ['FY','DJF','MAM', 'JJA', 'SON',])
    return df

def give_summary_1940_2100(wspd, verbose = False):
    
    data = np.zeros([5,10])
    s_y = [1940,1940,1980,2020,2060,]
    e_y = [2099,1979,2019,2059,2099]
    tseas = ['FY','DJF','MAM','JJA','SON']

    for y in range(0,5):
        for s in range(0,5):
            t1 = s_y[y]
            t2 = e_y[y]
            seas = tseas[s]

            if seas == 'FY':
                w = wspd.wspd10m.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-12-31'))\
                .groupby('time_counter.year').mean()
            else:
                w = wspd.wspd10m.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-12-31'))
                w = w.sel(time_counter=(w['time_counter.season'] == seas)).groupby('time_counter.year').mean()

            mk_slope, mk_p, lin_slope, lin_p = give_trends(w.year,w,t1,t2)
            mk_slope = mk_slope*10 #decadal
            lin_slope = lin_slope*10 #decadal
            if mk_p > 0.05:
                mk_slope = -999
            if lin_p > 0.05:
                lin_slope = -999  
            if verbose:
                print(f'winds, {seas}, {t1}-{t2}, MK: {mk_slope}, lin: {lin_slope}')
            
            data[s,y*2] = mk_slope
            data[s,y*2+1] = lin_slope
        if verbose:
            print()
        
    df = pd.DataFrame(data, columns=['MK-1940-2100','lin-1940-2100',\
                                     'MK-1940-1980','lin-1940-1980',\
                                     'MK-1980-2020','lin-1980-2020',\
                                     'MK-2020-2060','lin-2020-2060',\
                                     'MK-2060-2100','lin-2060-2100',\
                                  
                                   ],\
              index  = ['FY','DJF','MAM', 'JJA', 'SON',])
    return df



# trend tables, winds - historical/observational

- calculate each trend two ways, mann-kendall (sen's slope) and standard least-squares linear regression

## MT3 - historical wind speed in ERA5 and UKESM

    datasets: ukesm, tom
    periods: 1940-2020, 1940-1980, 1980-2020
    
or! do historical wind speed combined with sam

## ST2 - UKESM, TOM, SAM

    datasets: ukesm, tom, sam 
    periods: 1940-1957, 1957-1980, 1980-2000, 2000-2020
    

In [5]:
def give_summary_1940_2020_wind_WS(wspd, verbose = False):
    
    savenam = '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/wspdComponents/PlankTOMmask_regridrecalc.nc'
    cdomask = xr.open_dataset(savenam)
    cdomask
    tmask = cdomask.tmask

    data = np.zeros([5,8])
    s_y = [1940,1940,1980,1957]
    e_y = [2019,1979,2019,2019]
    tseas = ['FY','DJF','MAM','JJA','SON']

    for y in range(0,4):
        for s in range(0,5):
            t1 = s_y[y]
            t2 = e_y[y]
            seas = tseas[s]
            yr = t1
            yrend = t2
            if seas == 'FY':
                
                test = wspd.wspd10m.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-01-01'))
                test2 = test.isel(lat = slice(0,40)).weighted(tmask.isel(lat = slice(0,40))).\
                mean(dim = ['lat', 'lon']).groupby('time_counter.year').mean()
                
            else:
                
                test = wspd.wspd10m.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-01-01'))
                test2 = test.isel(lat = slice(0,40)).weighted(tmask.isel(lat = slice(0,40)))\
                    .mean(dim = ['lat', 'lon'])\
                    .sel(time_counter=(test['time_counter.season'] == seas))\
                    .groupby('time_counter.year').mean()

            mk_slope, mk_p, lin_slope, lin_p = give_trends(test2.year,test2.values,t1,t2)
            
            mk_slope = mk_slope*10 #decadal
            lin_slope = lin_slope*10 #decadal
            if mk_p > 0.05:
                mk_slope = -999
            if lin_p > 0.05:
                lin_slope = -999  
            if verbose:
                print(f'winds, {seas}, {t1}-{t2}, MK: {mk_slope}, lin: {lin_slope}')
            
            data[s,y*2] = mk_slope
            data[s,y*2+1] = lin_slope
        if verbose:
            print()
        
#     s_y = [1940,1940,1980,1957]
#     e_y = [2019,1979,2019,2019]
    df = pd.DataFrame(data, columns=[\
                                     'MK-1940-2019','lin',\
                                     'MK-1940-1979','lin',\
                                     'MK-1980-2019','lin',\
                                     'MK-1957-2019','lin',\
                                   ],\
              index  = ['FY','DJF','MAM', 'JJA', 'SON',])
    return df

In [6]:
def give_summary_1940_2020_wind(wspd, verbose = False):
    
    savenam = '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/wspdComponents/PlankTOMmask_regridrecalc.nc'
    cdomask = xr.open_dataset(savenam)
    cdomask
    tmask = cdomask.tmask

    data = np.zeros([5,10])
    s_y = [1957,1940,1957,1980,2000,]
    e_y = [2019,1956,1979,1999,2019]
    tseas = ['FY','DJF','MAM','JJA','SON']

    for y in range(0,5):
        for s in range(0,5):
            t1 = s_y[y]
            t2 = e_y[y]
            seas = tseas[s]
            yr = t1
            yrend = t2
            if seas == 'FY':
                
                test = wspd.wspd10m.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-01-01'))
                test2 = test.isel(lat = slice(0,40)).weighted(tmask.isel(lat = slice(0,40))).\
                mean(dim = ['lat', 'lon']).groupby('time_counter.year').mean()
                
            else:
                
                test = wspd.wspd10m.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-01-01'))
                test2 = test.isel(lat = slice(0,40)).weighted(tmask.isel(lat = slice(0,40)))\
                    .mean(dim = ['lat', 'lon'])\
                    .sel(time_counter=(test['time_counter.season'] == seas))\
                    .groupby('time_counter.year').mean()

            mk_slope, mk_p, lin_slope, lin_p = give_trends(test2.year,test2.values,t1,t2)
            
            mk_slope = mk_slope*10 #decadal
            lin_slope = lin_slope*10 #decadal
            if mk_p > 0.05:
                mk_slope = -999
            if lin_p > 0.05:
                lin_slope = -999  
            if verbose:
                print(f'winds, {seas}, {t1}-{t2}, MK: {mk_slope}, lin: {lin_slope}')
            
            data[s,y*2] = mk_slope
            data[s,y*2+1] = lin_slope
        if verbose:
            print()
        
    df = pd.DataFrame(data, columns=['MK-1957-2020','lin-1957-2020',\
                                     'MK-1940-1957','lin-1940-1957',\
                                     'MK-1957-1980','lin-1957-1980',\
                                     'MK-1980-2000','lin-1980-2000',\
                                     'MK-2000-2020','lin-2000-2020',\
                                   ],\
              index  = ['FY','DJF','MAM', 'JJA', 'SON',])
    return df

In [7]:
def give_summary_1957_2020_sam(wspd, verbose = False):
    
    savenam = '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/wspdComponents/PlankTOMmask_regridrecalc.nc'
    cdomask = xr.open_dataset(savenam)
    cdomask
    tmask = cdomask.tmask

    data = np.zeros([5,8])
    s_y = [1957,1957,1980,2000,]
    e_y = [2019,1979,1999,2019]
    tseas = ['FY','DJF','MAM','JJA','SON']

    for y in range(0,4):
        for s in range(0,5):
            t1 = s_y[y]
            t2 = e_y[y]
            seas = tseas[s]
            yr = t1
            yrend = t2

                
            if seas == 'FY':
                w = wspd.SAM.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-12-31'))\
                .groupby('time_counter.year').mean()
            else:
                w = wspd.SAM.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-12-31'))
                w = w.sel(time_counter=(w['time_counter.season'] == seas)).groupby('time_counter.year').mean()

            mk_slope, mk_p, lin_slope, lin_p = give_trends(w.year,w.values,t1,t2)
            
            mk_slope = mk_slope*10 #decadal
            lin_slope = lin_slope*10 #decadal
            if mk_p > 0.05:
                mk_slope = -999
            if lin_p > 0.05:
                lin_slope = -999  
            if verbose:
                print(f'winds, {seas}, {t1}-{t2}, MK: {mk_slope}, lin: {lin_slope}')
            
            data[s,y*2] = mk_slope
            data[s,y*2+1] = lin_slope
        if verbose:
            print()
        
    df = pd.DataFrame(data, columns=['MK-1957-2020','lin-1957-2020',\
                                     'MK-1957-1980','lin-1957-1980',\
                                     'MK-1980-2000','lin-1980-2000',\
                                     'MK-2000-2020','lin-2000-2020',\
                                   ],\
              index  = ['FY','DJF','MAM', 'JJA', 'SON',])
    return df

In [8]:
def make_yearlist_era(yrst, yrend):
    yrs = np.arange(yrst,yrend+1,1)
    ylist = []
    for i in range(0,len(yrs)):
        baseDir = '/gpfs/data/greenocean/software/products/windsFromComponents/ERA5_v202303_rawdat/daily'
        ty = f'{baseDir}/ERA5_y{yrs[i]}_wspd10m_seas_rg.nc'
        t2 = glob.glob(ty)
        ylist.append(t2[0])
    return ylist

def make_yearlist_ukesm(yrst, yrend, scen = '1A'):
    yrs = np.arange(yrst,yrend+1,1)
    ylist = []
    for i in range(0,len(yrs)):
        baseDir = '/gpfs/data/greenocean/software/products/windsFromComponents/symlinked_hrly_UKESM/daily'
        ty = f'{baseDir}/UKESM_{scen}_y{yrs[i]}_wspd10m_seas_rg.nc'
        t2 = glob.glob(ty)
        ylist.append(t2[0])
    return ylist

eralist = xr.open_mfdataset(make_yearlist_era(1940, 2020))
ukesmlist = xr.open_mfdataset(make_yearlist_ukesm(1940, 2020))
tsam = xr.open_dataset('../wspdComponents/sam/sam.nc')

In [9]:

era = give_summary_1940_2020_wind_WS(eralist, verbose = False)
print('era')
print(era)
ukesm = give_summary_1940_2020_wind_WS(ukesmlist, verbose = False)
print('ukesm')
print(ukesm)
sam = give_summary_1957_2020_sam(tsam, verbose = False)
print('sam')
print(sam)

era
     MK-1940-2019       lin  MK-1940-1979        lin  MK-1980-2019  \
FY       0.093670  0.085225    -999.00000 -999.00000      0.065352   
DJF      0.140515  0.135821    -999.00000 -999.00000      0.126631   
MAM      0.041285  0.043413      -0.09623   -0.09854   -999.000000   
JJA      0.087163  0.084328    -999.00000 -999.00000      0.068835   
SON      0.087436  0.077339    -999.00000 -999.00000   -999.000000   

            lin  MK-1957-2019       lin  
FY     0.062030      0.120673  0.118167  
DJF    0.121228      0.165700  0.158761  
MAM -999.000000      0.090765  0.093382  
JJA    0.065105      0.124977  0.124674  
SON -999.000000      0.106418  0.095854  
ukesm
     MK-1940-2019       lin  MK-1940-1979         lin  MK-1980-2019  \
FY       0.067988  0.066412      0.049294    0.043992      0.107221   
DJF      0.132909  0.128410   -999.000000 -999.000000      0.230262   
MAM      0.036939  0.036341   -999.000000 -999.000000      0.095156   
JJA      0.029183  0.025993   -99

## ST1 - historical poleward movement of winds in ERA5, UKESM

    datasets: ukesm, tom
    periods: 1940-2020, 1940-1980, 1980-2020

In [10]:
def give_summary_1940_2020_lat(wspd, verbose = False):
    
    data = np.zeros([4,6])
    s_y = [1940,1940,1980]
    e_y = [2019,1979,2019]
    tseas = ['DJF','MAM','JJA','SON']

    for y in range(0,3):
        for s in range(0,4):
            t1 = s_y[y]
            t2 = e_y[y]
            seas = tseas[s]

            if seas == 'FY':
                w = wspd.seas_latma.sel(times=slice(f'{t1}-01-01', f'{t2}-12-31'))\
                .groupby('times.year').mean()
            else:
                w = wspd.seas_latma.sel(times=slice(f'{t1}-01-01', f'{t2}-12-31'))
                w = w.sel(times=(w['times.season'] == seas)).groupby('times.year').mean()

            mk_slope, mk_p, lin_slope, lin_p = give_trends(w.year,w,t1,t2)
            mk_slope = mk_slope*10 #decadal
            lin_slope = lin_slope*10 #decadal
            if mk_p > 0.05:
                mk_slope = -999
            if lin_p > 0.05:
                lin_slope = -999  
            if verbose:
                print(f'winds, {seas}, {t1}-{t2}, MK: {mk_slope}, lin: {lin_slope}')
            
            data[s,y*2] = mk_slope
            data[s,y*2+1] = lin_slope
        if verbose:
            print()
        
    df = pd.DataFrame(data, columns=['MK-1940-2020','lin-1940-2020',\
                                     'MK-1940-1980','lin-1940-1980',\
                                     'MK-1980-2020','lin-1980-2020',\

                                   ],\
              index  = ['DJF','MAM', 'JJA', 'SON',])
    return df

def make_yearlist_era(yrst, yrend):
    yrs = np.arange(yrst,yrend+1,1)
    ylist = []
    for i in range(0,len(yrs)):
        baseDir = '/gpfs/data/greenocean/software/products/windsFromComponents/ERA5_v202303_rawdat/daily'
        ty = f'{baseDir}/10m_windspeed_ERA5_y{yrs[i]}_zonal-seas.nc'
        t2 = glob.glob(ty)
        ylist.append(t2[0])
    return ylist

def make_yearlist_ukesm(yrst, yrend, scen = '1A'):
    yrs = np.arange(yrst,yrend+1,1)
    ylist = []
    for i in range(0,len(yrs)):
        baseDir = '/gpfs/data/greenocean/software/products/windsFromComponents/symlinked_hrly_UKESM/daily'
        ty = f'{baseDir}/UKESM_{scen}_y{yrs[i]}_zonal-seas.nc'
        t2 = glob.glob(ty)
        ylist.append(t2[0])
    return ylist

eraw= xr.open_mfdataset(make_yearlist_era(1940, 2020))
ukesmw = xr.open_mfdataset(make_yearlist_ukesm(1940, 2020))


era = give_summary_1940_2020_lat(eraw, verbose = False)
print('era')
print(era)
print()
ukesm = give_summary_1940_2020_lat(ukesmw, verbose = False)
print('ukesm')
print(ukesm)

era
     MK-1940-2020  lin-1940-2020  MK-1940-1980  lin-1940-1980  MK-1980-2020  \
DJF     -0.198156      -0.194635        -999.0         -999.0        -999.0   
MAM      0.108696       0.121054        -999.0         -999.0        -999.0   
JJA      0.149395       0.150074        -999.0         -999.0        -999.0   
SON   -999.000000    -999.000000        -999.0         -999.0        -999.0   

     lin-1980-2020  
DJF      -0.300484  
MAM    -999.000000  
JJA    -999.000000  
SON    -999.000000  

ukesm
     MK-1940-2020  lin-1940-2020  MK-1940-1980  lin-1940-1980  MK-1980-2020  \
DJF     -0.354273      -0.362174        -999.0         -999.0     -0.647487   
MAM   -999.000000    -999.000000        -999.0         -999.0   -999.000000   
JJA   -999.000000    -999.000000        -999.0         -999.0     -0.662393   
SON     -0.215741      -0.186278        -999.0         -999.0   -999.000000   

     lin-1980-2020  
DJF      -0.624098  
MAM    -999.000000  
JJA      -0.541974  
SON    -

# trend tables, winds speed, forward


    datasets: ukesm, all 6
    periods: 1940-2100, 1940-1980, 1980-2020, 2020-2060, 2060-2100
    periods: 1950-2100, 1950-2000, 2000-2050, 2050-2100

In [11]:
baseDir = '//gpfs/data/greenocean/software/products/windsFromComponents/symlinked_hrly_UKESM/daily/'

wspd_PI = xr.open_dataset(f'{baseDir}/UKESM_PI_wspd10m_overwater-50S-mean.nc')
wspd_1A = xr.open_dataset(f'{baseDir}/UKESM_1A_wspd10m_overwater-50S-mean.nc')
wspd_2A = xr.open_dataset(f'{baseDir}/UKESM_2A_wspd10m_overwater-50S-mean.nc')
wspd_3A = xr.open_dataset(f'{baseDir}/UKESM_3A_wspd10m_overwater-50S-mean.nc')
wspd_1B = xr.open_dataset(f'{baseDir}/UKESM_1B_wspd10m_overwater-50S-mean.nc')
wspd_2B = xr.open_dataset(f'{baseDir}/UKESM_2B_wspd10m_overwater-50S-mean.nc')
wspd_3B = xr.open_dataset(f'{baseDir}/UKESM_3B_wspd10m_overwater-50S-mean.nc')

In [12]:
df_1A = give_summary_1950_2100(wspd_1A)
print('')
print('1A')
print(df_1A)
df_2A = give_summary_1950_2100(wspd_2A)
print('')
print('2A')
print(df_2A)
df_3A = give_summary_1950_2100(wspd_3A)
print('')
print('3A')
print(df_3A)
df_1B = give_summary_1950_2100(wspd_1B)
print('')
print('1B')
print(df_1B)
df_2B = give_summary_1950_2100(wspd_2B)
print('')
print('2B')
print(df_2B)
df_3B = give_summary_1950_2100(wspd_3B)
print('')
print('3B')
print(df_3B)



1A
     MK-1950-2100  lin-1950-2100  MK-1950-2000  lin-1950-2000  MK-2000-2050  \
FY       0.029650       0.027540      0.066177       0.063205        -999.0   
DJF      0.035160       0.035193      0.122595       0.121807        -999.0   
MAM      0.027223       0.024876   -999.000000    -999.000000        -999.0   
JJA      0.024096       0.023339   -999.000000    -999.000000        -999.0   
SON      0.029604       0.026753      0.099119       0.093785        -999.0   

     lin-2000-2050  MK-2050-2100  lin-2050-2100  
FY          -999.0   -999.000000    -999.000000  
DJF         -999.0     -0.081208      -0.076564  
MAM         -999.0   -999.000000    -999.000000  
JJA         -999.0   -999.000000    -999.000000  
SON         -999.0   -999.000000    -999.000000  

2A
     MK-1950-2100  lin-1950-2100  MK-1950-2000  lin-1950-2000  MK-2000-2050  \
FY       0.028498       0.028206        -999.0         -999.0      0.033615   
DJF      0.034613       0.035567        -999.0         -999

In [13]:
df_1A = give_summary_1940_2100(wspd_1A)
print('')
print('1A')
print(df_1A)
df_2A = give_summary_1940_2100(wspd_2A)
print('')
print('2A')
print(df_2A)
df_3A = give_summary_1940_2100(wspd_3A)
print('')
print('3A')
print(df_3A)
df_1B = give_summary_1940_2100(wspd_1B)
print('')
print('1B')
print(df_1B)
df_2B = give_summary_1940_2100(wspd_2B)
print('')
print('2B')
print(df_2B)
df_3B = give_summary_1940_2100(wspd_3B)
print('')
print('3B')
print(df_3B)



1A
     MK-1940-2100  lin-1940-2100  MK-1940-1980  lin-1940-1980  MK-1980-2020  \
FY       0.029650       0.027540      0.061568         -999.0      0.098746   
DJF      0.035160       0.035193   -999.000000         -999.0      0.212102   
MAM      0.027223       0.024876   -999.000000         -999.0      0.081165   
JJA      0.024096       0.023339   -999.000000         -999.0      0.084622   
SON      0.029604       0.026753   -999.000000         -999.0   -999.000000   

     lin-1980-2020  MK-2020-2060  lin-2020-2060  MK-2060-2100  lin-2060-2100  
FY        0.092424        -999.0    -999.000000     -0.041355    -999.000000  
DJF       0.192713        -999.0      -0.097618     -0.147273      -0.144527  
MAM       0.079557        -999.0    -999.000000   -999.000000    -999.000000  
JJA       0.082229        -999.0    -999.000000   -999.000000    -999.000000  
SON    -999.000000        -999.0    -999.000000   -999.000000    -999.000000  

2A
     MK-1940-2100  lin-1940-2100  MK-1940-1

# trend tables, wind polar position, forward


    datasets: ukesm, all 6
    periods: 1940-2100, 1940-1980, 1980-2020, 2020-2060, 2060-2100

In [14]:
def make_yearlist_ukesm(yrst, yrend, scen = '1A'):
    yrs = np.arange(yrst,yrend+1,1)
    ylist = []
    for i in range(0,len(yrs)):
        baseDir = '/gpfs/data/greenocean/software/products/windsFromComponents/symlinked_hrly_UKESM/daily'
        ty = f'{baseDir}/UKESM_{scen}_y{yrs[i]}_zonal-seas.nc'
        t2 = glob.glob(ty)
        ylist.append(t2[0])
    return ylist

#eralist = make_yearlist_era(1940, 2020)
uk_1A = xr.open_mfdataset(make_yearlist_ukesm(1940, 2100))
uk_1A = xr.open_mfdataset(make_yearlist_ukesm(1940, 2100, '1A'))
uk_2A = xr.open_mfdataset(make_yearlist_ukesm(1940, 2100, '2A'))
uk_3A = xr.open_mfdataset(make_yearlist_ukesm(1940, 2100, '3A'))
uk_1B = xr.open_mfdataset(make_yearlist_ukesm(1940, 2100, '1B'))
uk_2B = xr.open_mfdataset(make_yearlist_ukesm(1940, 2100, '2B'))
uk_3B = xr.open_mfdataset(make_yearlist_ukesm(1940, 2100, '3B'))

def give_summary_1940_2100_lat(wspd, verbose = False):
    
    data = np.zeros([4,10])
    s_y = [1940,1940,1980,2020,2060]
    e_y = [2099,1979,2019,2059,2099]
    tseas = ['DJF','MAM','JJA','SON']

    for y in range(0,5):
        for s in range(0,4):
            t1 = s_y[y]
            t2 = e_y[y]
            seas = tseas[s]

            if seas == 'FY':
                w = wspd.seas_latma.sel(times=slice(f'{t1}-01-01', f'{t2}-12-31'))\
                .groupby('times.year').mean()
            else:
                w = wspd.seas_latma.sel(times=slice(f'{t1}-01-01', f'{t2}-12-31'))
                w = w.sel(times=(w['times.season'] == seas)).groupby('times.year').mean()

            mk_slope, mk_p, lin_slope, lin_p = give_trends(w.year,w,t1,t2)
            mk_slope = mk_slope*10 #decadal
            lin_slope = lin_slope*10 #decadal
            if mk_p > 0.05:
                mk_slope = -999
            if lin_p > 0.05:
                lin_slope = -999  
            if verbose:
                print(f'winds, {seas}, {t1}-{t2}, MK: {mk_slope}, lin: {lin_slope}')
            
            data[s,y*2] = mk_slope
            data[s,y*2+1] = lin_slope
        if verbose:
            print()
        
    df = pd.DataFrame(data, columns=['MK-1940-2100','lin-1940-2100',\
                                     'MK-1940-1980','lin-1940-1980',\
                                     'MK-1980-2020','lin-1980-2020',\
                                     'MK-2020-2060','lin-2020-2060',\
                                     'MK-2060-2100','lin-2060-2100',\

                                   ],\
              index  = ['DJF','MAM', 'JJA', 'SON',])
    return df

df_1A = give_summary_1940_2100_lat(uk_1A)
print('')
print('1A')
print(df_1A)
df_2A = give_summary_1940_2100_lat(uk_2A)
print('')
print('2A')
print(df_2A)
df_3A = give_summary_1940_2100_lat(uk_3A)
print('')
print('3A')
print(df_3A)
df_1B = give_summary_1940_2100_lat(uk_1B)
print('')
print('1B')
print(df_1B)
df_2B = give_summary_1940_2100_lat(uk_2B)
print('')
print('2B')
print(df_2B)
df_3B = give_summary_1940_2100_lat(uk_3B)
print('')
print('3B')
print(df_3B)



1A
     MK-1940-2100  lin-1940-2100  MK-1940-1980  lin-1940-1980  MK-1980-2020  \
DJF     -0.123098      -0.117581        -999.0         -999.0     -0.647487   
MAM     -0.092407      -0.086261        -999.0         -999.0   -999.000000   
JJA     -0.075075      -0.073300        -999.0         -999.0     -0.662393   
SON     -0.090735      -0.088297        -999.0         -999.0   -999.000000   

     lin-1980-2020  MK-2020-2060  lin-2020-2060  MK-2060-2100  lin-2060-2100  
DJF      -0.624098        -999.0         -999.0      0.712431       0.710454  
MAM    -999.000000        -999.0         -999.0   -999.000000    -999.000000  
JJA      -0.541974        -999.0         -999.0   -999.000000    -999.000000  
SON    -999.000000        -999.0         -999.0   -999.000000    -999.000000  

2A
     MK-1940-2100  lin-1940-2100  MK-1940-1980  lin-1940-1980  MK-1980-2020  \
DJF     -0.110090      -0.108938        -999.0         -999.0        -999.0   
MAM     -0.092765      -0.091782        -99

In [15]:
def give_summary_1950_2100_lat(wspd, verbose = False):
    
    data = np.zeros([4,8])
    s_y = [1950,1950,2000,2050]
    e_y = [2099,1999,2049,2099]
    tseas = ['DJF','MAM','JJA','SON']

    for y in range(0,4):
        for s in range(0,4):
            t1 = s_y[y]
            t2 = e_y[y]
            seas = tseas[s]

            if seas == 'FY':
                w = wspd.seas_latma.sel(times=slice(f'{t1}-01-01', f'{t2}-12-31'))\
                .groupby('times.year').mean()
            else:
                w = wspd.seas_latma.sel(times=slice(f'{t1}-01-01', f'{t2}-12-31'))
                w = w.sel(times=(w['times.season'] == seas)).groupby('times.year').mean()

            mk_slope, mk_p, lin_slope, lin_p = give_trends(w.year,w,t1,t2)
            mk_slope = mk_slope*10 #decadal
            lin_slope = lin_slope*10 #decadal
            if mk_p > 0.05:
                mk_slope = -999
            if lin_p > 0.05:
                lin_slope = -999  
            if verbose:
                print(f'winds, {seas}, {t1}-{t2}, MK: {mk_slope}, lin: {lin_slope}')
            
            data[s,y*2] = mk_slope
            data[s,y*2+1] = lin_slope
        if verbose:
            print()
        
    df = pd.DataFrame(data, columns=['MK-1950-2100','lin-1950-2100',\
                                     'MK-1950-2000','lin-1950-2000',\
                                     'MK-2000-2050','lin-2000-2050',\
                                     'MK-2050-2100','lin-2050-2100',\
             

                                   ],\
              index  = ['DJF','MAM', 'JJA', 'SON',])
    return df

df_1A = give_summary_1950_2100_lat(uk_1A)
print('')
print('1A')
print(df_1A)
df_2A = give_summary_1950_2100_lat(uk_2A)
print('')
print('2A')
print(df_2A)
df_3A = give_summary_1950_2100_lat(uk_3A)
print('')
print('3A')
print(df_3A)
df_1B = give_summary_1950_2100_lat(uk_1B)
print('')
print('1B')
print(df_1B)
df_2B = give_summary_1950_2100_lat(uk_2B)
print('')
print('2B')
print(df_2B)
df_3B = give_summary_1950_2100_lat(uk_3B)
print('')
print('3B')
print(df_3B)



1A
     MK-1950-2100  lin-1950-2100  MK-1950-2000  lin-1950-2000  MK-2000-2050  \
DJF     -0.111111      -0.107937     -0.353909      -0.352045        -999.0   
MAM     -0.098413      -0.093464   -999.000000    -999.000000        -999.0   
JJA     -0.088889      -0.080538   -999.000000    -999.000000        -999.0   
SON     -0.080808      -0.071699   -999.000000    -999.000000        -999.0   

     lin-2000-2050  MK-2050-2100  lin-2050-2100  
DJF         -999.0      0.391667       0.309735  
MAM         -999.0   -999.000000    -999.000000  
JJA         -999.0   -999.000000    -999.000000  
SON         -999.0   -999.000000    -999.000000  

2A
     MK-1950-2100  lin-1950-2100  MK-1950-2000  lin-1950-2000  MK-2000-2050  \
DJF     -0.124444      -0.124826        -999.0         -999.0        -999.0   
MAM     -0.105475      -0.108238        -999.0         -999.0        -999.0   
JJA     -0.144703      -0.142594        -999.0         -999.0        -999.0   
SON   -999.000000    -999.0000

## trend tables for MLD, OT, SST

    datasets: ukesm, and tom, all 6
    periods: 1950-2100, 1950-2000, 2000-2050, 2050-2100

In [16]:
mld_ukesm_1A = xr.open_dataset(f'./extracted-summary/somxl010_ukesm_1A.nc')
mld_ukesm_1B = xr.open_dataset(f'./extracted-summary/somxl010_ukesm_1B.nc')
mld_ukesm_2A = xr.open_dataset(f'./extracted-summary/somxl010_ukesm_2A.nc')
mld_ukesm_2B = xr.open_dataset(f'./extracted-summary/somxl010_ukesm_2B.nc')
mld_ukesm_3A = xr.open_dataset(f'./extracted-summary/somxl010_ukesm_3A.nc')
mld_ukesm_3B = xr.open_dataset(f'./extracted-summary/somxl010_ukesm_3B.nc')

mld_tom_1A = xr.open_dataset(f'./extracted-summary/somxl010_tom_1AA6.nc')
mld_tom_1B = xr.open_dataset(f'./extracted-summary/somxl010_tom_1BA6.nc')
mld_tom_2A = xr.open_dataset(f'./extracted-summary/somxl010_tom_2AA6.nc')
mld_tom_2B = xr.open_dataset(f'./extracted-summary/somxl010_tom_2BA6.nc')
mld_tom_3A = xr.open_dataset(f'./extracted-summary/somxl010_tom_3AA6.nc')
mld_tom_3B = xr.open_dataset(f'./extracted-summary/somxl010_tom_3BA6.nc')

sst_ukesm_1A = xr.open_dataset(f'./extracted-summary/SST_ukesm_1A.nc')
sst_ukesm_1B = xr.open_dataset(f'./extracted-summary/SST_ukesm_1B.nc')
sst_ukesm_2A = xr.open_dataset(f'./extracted-summary/SST_ukesm_2A.nc')
sst_ukesm_2B = xr.open_dataset(f'./extracted-summary/SST_ukesm_2B.nc')
sst_ukesm_3A = xr.open_dataset(f'./extracted-summary/SST_ukesm_3A.nc')
sst_ukesm_3B = xr.open_dataset(f'./extracted-summary/SST_ukesm_3B.nc')

sst_tom_1A = xr.open_dataset(f'./extracted-summary/SST_tom_1AA6.nc')
sst_tom_1B = xr.open_dataset(f'./extracted-summary/SST_tom_1BA6.nc')
sst_tom_2A = xr.open_dataset(f'./extracted-summary/SST_tom_2AA6.nc')
sst_tom_2B = xr.open_dataset(f'./extracted-summary/SST_tom_2BA6.nc')
sst_tom_3A = xr.open_dataset(f'./extracted-summary/SST_tom_3AA6.nc')
sst_tom_3B = xr.open_dataset(f'./extracted-summary/SST_tom_3BA6.nc')

In [17]:
def give_summary_1950_2100_mld(wspd, verbose = False):
    
    data = np.zeros([5,8])
    s_y = [1950,1950,2000,2050]
    e_y = [2099,1999,2049,2099]
    tseas = ['FY','DJF','MAM','JJA','SON']

    for y in range(0,4):
        for s in range(0,5):
            t1 = s_y[y]
            t2 = e_y[y]
            seas = tseas[s]

            if seas == 'FY':
                w = wspd.somxl010.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-12-31'))\
                .groupby('time_counter.year').mean()
            else:
                w = wspd.somxl010.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-12-31'))
                w = w.sel(time_counter=(w['time_counter.season'] == seas)).groupby('time_counter.year').mean()

            mk_slope, mk_p, lin_slope, lin_p = give_trends(w.year,w,t1,t2)
            mk_slope = mk_slope*10 #decadal
            lin_slope = lin_slope*10 #decadal
            if mk_p > 0.05:
                mk_slope = -999
            if lin_p > 0.05:
                lin_slope = -999  
            if verbose:
                print(f'winds, {seas}, {t1}-{t2}, MK: {mk_slope}, lin: {lin_slope}')
            
            data[s,y*2] = mk_slope
            data[s,y*2+1] = lin_slope
        if verbose:
            print()
        
    df = pd.DataFrame(data, columns=['MK-1950-2100','lin-1950-2100',\
                                   'MK-1950-2000','lin-1950-2000',\
                                   'MK-2000-2050','lin-2000-2050',\
                                   'MK-2050-2100','lin-2050-2100',\
                                   ],\
              index  = ['FY','DJF','MAM', 'JJA', 'SON',])
    return df

In [18]:
df_1A = give_summary_1950_2100_mld(mld_ukesm_1A)
print('UKESM ')
print('UKESM 1A')
print(df_1A)
df_2A = give_summary_1950_2100_mld(mld_ukesm_2A)
print('UKESM ')
print('UKESM 2A')
print(df_2A)
df_3A = give_summary_1950_2100_mld(mld_ukesm_3A)
print('UKESM ')
print('UKESM 3A')
print(df_3A)
df_1B = give_summary_1950_2100_mld(mld_ukesm_1B)
print('UKESM ')
print('UKESM 1B')
print(df_1B)
df_2B = give_summary_1950_2100_mld(mld_ukesm_2B)
print('UKESM ')
print('UKESM 2B')
print(df_2B)
df_3B = give_summary_1950_2100_mld(mld_ukesm_3B)
print('UKESM ')
print('UKESM 3B')
print(df_3B)

df_1A = give_summary_1950_2100_mld(mld_tom_1A)
print('tom ')
print('tom 1A')
print(df_1A)
df_2A = give_summary_1950_2100_mld(mld_tom_2A)
print('tom ')
print('tom 2A')
print(df_2A)
df_3A = give_summary_1950_2100_mld(mld_tom_3A)
print('tom ')
print('tom 3A')
print(df_3A)
df_1B = give_summary_1950_2100_mld(mld_tom_1B)
print('tom ')
print('tom 1B')
print(df_1B)
df_2B = give_summary_1950_2100_mld(mld_tom_2B)
print('tom ')
print('tom 2B')
print(df_2B)
df_3B = give_summary_1950_2100_mld(mld_tom_3B)
print('tom ')
print('tom 3B')
print(df_3B)

UKESM 
UKESM 1A
     MK-1950-2100  lin-1950-2100  MK-1950-2000  lin-1950-2000  MK-2000-2050  \
FY      -0.208200      -0.213274      1.546621       1.496040     -1.599353   
DJF   -999.000000    -999.000000      1.240822       1.324724     -1.059141   
MAM   -999.000000    -999.000000      1.163086       1.123183   -999.000000   
JJA     -0.314265      -0.298781      1.502059       1.401421     -2.109860   
SON     -0.480496      -0.497682      2.232608       2.134832     -2.408289   

     lin-2000-2050  MK-2050-2100  lin-2050-2100  
FY       -1.542798   -999.000000    -999.000000  
DJF      -0.981855     -0.833878      -0.794539  
MAM    -999.000000   -999.000000    -999.000000  
JJA      -1.897011   -999.000000    -999.000000  
SON      -2.557352   -999.000000    -999.000000  
UKESM 
UKESM 2A
     MK-1950-2100  lin-1950-2100  MK-1950-2000  lin-1950-2000  MK-2000-2050  \
FY      -0.144533      -0.153525   -999.000000      -0.640251     -0.530193   
DJF   -999.000000    -999.000000   

tom 
tom 3B
     MK-1950-2100  lin-1950-2100  MK-1950-2000  lin-1950-2000  MK-2000-2050  \
FY      -0.232402      -0.135485      1.346386       1.685618   -999.000000   
DJF      0.458709       0.447924      0.929517       1.058499      0.412383   
MAM      0.179187       0.207035      0.771827       0.865290   -999.000000   
JJA     -0.695677      -0.542022      1.601355       2.138677     -0.724896   
SON     -0.800698      -0.654876      2.322238       2.680006     -1.115171   

     lin-2000-2050  MK-2050-2100  lin-2050-2100  
FY     -999.000000     -0.465462      -0.511781  
DJF       0.488342   -999.000000    -999.000000  
MAM    -999.000000   -999.000000    -999.000000  
JJA      -0.776732     -1.035280      -1.073603  
SON      -1.101366     -1.335481      -1.236245  


### sst

In [19]:
def give_summary_1950_2100_sst(wspd, verbose = False):
    
    data = np.zeros([5,8])
    s_y = [1950,1950,2000,2050]
    e_y = [2099,1999,2049,2099]
    tseas = ['FY','DJF','MAM','JJA','SON']

    for y in range(0,4):
        for s in range(0,5):
            t1 = s_y[y]
            t2 = e_y[y]
            seas = tseas[s]

            if seas == 'FY':
                w = wspd.tos.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-12-31'))\
                .groupby('time_counter.year').mean()
            else:
                w = wspd.tos.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-12-31'))
                w = w.sel(time_counter=(w['time_counter.season'] == seas)).groupby('time_counter.year').mean()

            mk_slope, mk_p, lin_slope, lin_p = give_trends(w.year,w,t1,t2)
            mk_slope = mk_slope*10 #decadal
            lin_slope = lin_slope*10 #decadal
            if mk_p > 0.05:
                mk_slope = -999
            if lin_p > 0.05:
                lin_slope = -999  
            if verbose:
                print(f'winds, {seas}, {t1}-{t2}, MK: {mk_slope}, lin: {lin_slope}')
            
            data[s,y*2] = mk_slope
            data[s,y*2+1] = lin_slope
        if verbose:
            print()
        
    df = pd.DataFrame(data, columns=['MK-1950-2100','lin-1950-2100',\
                                   'MK-1950-2000','lin-1950-2000',\
                                   'MK-2000-2050','lin-2000-2050',\
                                   'MK-2050-2100','lin-2050-2100',\
                                   ],\
              index  = ['FY','DJF','MAM', 'JJA', 'SON',])
    return df

In [20]:
df_1A = give_summary_1950_2100_sst(sst_ukesm_1A)
print('UKESM ')
print('UKESM 1A')
print(df_1A)
df_2A = give_summary_1950_2100_sst(sst_ukesm_2A)
print('UKESM ')
print('UKESM 2A')
print(df_2A)
df_3A = give_summary_1950_2100_sst(sst_ukesm_3A)
print('UKESM ')
print('UKESM 3A')
print(df_3A)
df_1B = give_summary_1950_2100_sst(sst_ukesm_1B)
print('UKESM ')
print('UKESM 1B')
print(df_1B)
df_2B = give_summary_1950_2100_sst(sst_ukesm_2B)
print('UKESM ')
print('UKESM 2B')
print(df_2B)
df_3B = give_summary_1950_2100_sst(sst_ukesm_3B)
print('UKESM ')
print('UKESM 3B')
print(df_3B)

df_1A = give_summary_1950_2100_sst(sst_tom_1A)
print('tom ')
print('tom 1A')
print(df_1A)
df_2A = give_summary_1950_2100_sst(sst_tom_2A)
print('tom ')
print('tom 2A')
print(df_2A)
df_3A = give_summary_1950_2100_sst(sst_tom_3A)
print('tom ')
print('tom 3A')
print(df_3A)
df_1B = give_summary_1950_2100_sst(sst_tom_1B)
print('tom ')
print('tom 1B')
print(df_1B)
df_2B = give_summary_1950_2100_sst(sst_tom_2B)
print('tom ')
print('tom 2B')
print(df_2B)
df_3B = give_summary_1950_2100_sst(sst_tom_3B)
print('tom ')
print('tom 3B')
print(df_3B)

UKESM 
UKESM 1A
     MK-1950-2100  lin-1950-2100  MK-1950-2000  lin-1950-2000  MK-2000-2050  \
FY       0.083227       0.084269   -999.000000      -0.028422      0.190671   
DJF      0.090969       0.090402     -0.053835      -0.053978      0.220070   
MAM      0.087838       0.088608   -999.000000    -999.000000      0.201399   
JJA      0.080437       0.080931   -999.000000    -999.000000      0.172458   
SON      0.077113       0.077136   -999.000000      -0.016220      0.167395   

     lin-2000-2050  MK-2050-2100  lin-2050-2100  
FY        0.182485        -999.0         -999.0  
DJF       0.212071        -999.0         -999.0  
MAM       0.191381        -999.0         -999.0  
JJA       0.166610        -999.0         -999.0  
SON       0.159878        -999.0         -999.0  
UKESM 
UKESM 2A
     MK-1950-2100  lin-1950-2100  MK-1950-2000  lin-1950-2000  MK-2000-2050  \
FY       0.066993       0.067158   -999.000000         -999.0      0.097879   
DJF      0.071255       0.072203   

tom 
tom 3B
     MK-1950-2100  lin-1950-2100  MK-1950-2000  lin-1950-2000  MK-2000-2050  \
FY       0.156385       0.147998        -999.0         -999.0      0.180690   
DJF      0.159174       0.152559        -999.0         -999.0      0.193318   
MAM      0.162419       0.155793        -999.0         -999.0      0.185790   
JJA      0.154014       0.142879        -999.0         -999.0      0.176463   
SON      0.151108       0.140761        -999.0         -999.0      0.173771   

     lin-2000-2050  MK-2050-2100  lin-2050-2100  
FY        0.180861      0.238277       0.236484  
DJF       0.187565      0.246062       0.246617  
MAM       0.184641      0.250568       0.249290  
JJA       0.176952      0.226547       0.225216  
SON       0.174286      0.221721       0.224815  


## overturning

In [21]:
def make_yearlist_tom(yrst, yrend, tscen, \
                  tdir = '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/oceanFields/max_OT/'):
    yrs = np.arange(yrst,yrend+1,1)
    ylist = []
    for i in range(0,len(yrs)):
        yr = yrs[i]
        ty = f'{tdir}/seas_OT_{yr}_{tscen}-TOMA6.nc'
        t2 = glob.glob(ty)
        #print(t2)
        ylist.append(t2[0])
    return ylist

ot_tom_1A = xr.open_mfdataset(make_yearlist_tom(1950,2099,'1A'))
ot_tom_2A = xr.open_mfdataset(make_yearlist_tom(1950,2099,'2A'))
ot_tom_3A = xr.open_mfdataset(make_yearlist_tom(1950,2099,'3A'))
ot_tom_1B = xr.open_mfdataset(make_yearlist_tom(1950,2099,'1B'))
ot_tom_2B = xr.open_mfdataset(make_yearlist_tom(1950,2099,'2B'))
ot_tom_3B = xr.open_mfdataset(make_yearlist_tom(1950,2099,'3B'))

def make_yearlist(yrst, yrend, tscen, \
                  tdir = '/gpfs/home/mep22dku/scratch/SOZONE/windAnalyis/oceanFields/max_OT_s50/'):
    yrs = np.arange(yrst,yrend+1,1)
    ylist = []
    for i in range(0,len(yrs)):
        yr = yrs[i]
        ty = f'{tdir}/seas_OT_{yr}_{tscen}.nc'
        t2 = glob.glob(ty)
        #print(t2)
        ylist.append(t2[0])
    return ylist

ot_ukesm_1A = xr.open_mfdataset(make_yearlist(1950,2099,'1A'))
ot_ukesm_2A = xr.open_mfdataset(make_yearlist(1950,2099,'2A'))
ot_ukesm_3A = xr.open_mfdataset(make_yearlist(1950,2099,'3A'))
ot_ukesm_1B = xr.open_mfdataset(make_yearlist(1950,2099,'1B'))
ot_ukesm_2B = xr.open_mfdataset(make_yearlist(1950,2099,'2B'))
ot_ukesm_3B = xr.open_mfdataset(make_yearlist(1950,2099,'3B'))

In [22]:
def give_summary_1950_2100_ot(wspd, verbose = False):
    
    data = np.zeros([5,8])
    s_y = [1950,1950,2000,2050]
    e_y = [2099,1999,2049,2099]
    tseas = ['FY','DJF','MAM','JJA','SON']

    for y in range(0,4):
        for s in range(0,5):
            t1 = s_y[y]
            t2 = e_y[y]
            seas = tseas[s]

            if seas == 'FY':
                w = wspd.max_OT_yr.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-12-31'))\
                .groupby('time_counter.year').mean()
            else:
                w = wspd.max_OT.sel(time_counter=slice(f'{t1}-01-01', f'{t2}-12-31'))
                w = w.sel(time_counter=(w['time_counter.season'] == seas)).groupby('time_counter.year').mean()

            mk_slope, mk_p, lin_slope, lin_p = give_trends(w.year,w,t1,t2)
            mk_slope = mk_slope*10 #decadal
            lin_slope = lin_slope*10 #decadal
            if mk_p > 0.05:
                mk_slope = -999
            if lin_p > 0.05:
                lin_slope = -999  
            if verbose:
                print(f'winds, {seas}, {t1}-{t2}, MK: {mk_slope}, lin: {lin_slope}')
            
            data[s,y*2] = mk_slope
            data[s,y*2+1] = lin_slope
        if verbose:
            print()
        
    df = pd.DataFrame(data, columns=['MK-1950-2100','lin-1950-2100',\
                                   'MK-1950-2000','lin-1950-2000',\
                                   'MK-2000-2050','lin-2000-2050',\
                                   'MK-2050-2100','lin-2050-2100',\
                                   ],\
              index  = ['FY','DJF','MAM', 'JJA', 'SON',])
    return df

In [23]:
df_1A = give_summary_1950_2100_ot(ot_ukesm_1A)
print('UKESM ')
print('UKESM 1A')
print(df_1A)
df_2A = give_summary_1950_2100_ot(ot_ukesm_2A)
print('UKESM ')
print('UKESM 2A')
print(df_2A)
df_3A = give_summary_1950_2100_ot(ot_ukesm_3A)
print('UKESM ')
print('UKESM 3A')
print(df_3A)
df_1B = give_summary_1950_2100_ot(ot_ukesm_1B)
print('UKESM ')
print('UKESM 1B')
print(df_1B)
df_2B = give_summary_1950_2100_ot(ot_ukesm_2B)
print('UKESM ')
print('UKESM 2B')
print(df_2B)
df_3B = give_summary_1950_2100_ot(ot_ukesm_3B)
print('UKESM ')
print('UKESM 3B')
print(df_3B)

df_1A = give_summary_1950_2100_ot(ot_tom_1A)
print('tom ')
print('tom 1A')
print(df_1A)
df_2A = give_summary_1950_2100_ot(ot_tom_2A)
print('tom ')
print('tom 2A')
print(df_2A)
df_3A = give_summary_1950_2100_ot(ot_tom_3A)
print('tom ')
print('tom 3A')
print(df_3A)
df_1B = give_summary_1950_2100_ot(ot_tom_1B)
print('tom ')
print('tom 1B')
print(df_1B)
df_2B = give_summary_1950_2100_ot(ot_tom_2B)
print('tom ')
print('tom 2B')
print(df_2B)
df_3B = give_summary_1950_2100_ot(ot_tom_3B)
print('tom ')
print('tom 3B')
print(df_3B)

UKESM 
UKESM 1A
     MK-1950-2100  lin-1950-2100  MK-1950-2000  lin-1950-2000  MK-2000-2050  \
FY       0.261545       0.268534      0.686851       0.773721        -999.0   
DJF      0.230068       0.263762      1.316125       1.400092        -999.0   
MAM      0.258592       0.288962   -999.000000    -999.000000        -999.0   
JJA      0.401249       0.370736   -999.000000    -999.000000        -999.0   
SON      0.326836       0.277712      0.882212       0.971834        -999.0   

     lin-2000-2050  MK-2050-2100  lin-2050-2100  
FY          -999.0        -999.0         -999.0  
DJF         -999.0        -999.0         -999.0  
MAM         -999.0        -999.0         -999.0  
JJA         -999.0        -999.0         -999.0  
SON         -999.0        -999.0         -999.0  
UKESM 
UKESM 2A
     MK-1950-2100  lin-1950-2100  MK-1950-2000  lin-1950-2000  MK-2000-2050  \
FY       0.330678       0.344552   -999.000000    -999.000000      0.437065   
DJF      0.283872       0.303801   

tom 
tom 3B
     MK-1950-2100  lin-1950-2100  MK-1950-2000  lin-1950-2000  MK-2000-2050  \
FY       0.547979       0.551555      0.670867       0.676060   -999.000000   
DJF      0.575125       0.566434      0.945209       0.947995   -999.000000   
MAM      0.505489       0.505487   -999.000000    -999.000000   -999.000000   
JJA      0.468052       0.456472   -999.000000    -999.000000      0.558318   
SON      0.573019       0.561218      0.830902       0.787038   -999.000000   

     lin-2000-2050  MK-2050-2100  lin-2050-2100  
FY     -999.000000      0.397014       0.445825  
DJF    -999.000000   -999.000000    -999.000000  
MAM    -999.000000   -999.000000       0.467483  
JJA       0.448646      0.816940       0.762591  
SON    -999.000000   -999.000000    -999.000000  
